几个超参数
每次测试迭代的次数，

In [1]:
import caffe
import numpy as np
from numpy import array
from numpy.random import normal
from matplotlib import pyplot
import os, sys
import struct

caffe.set_device(0)  
caffe.set_mode_gpu()
prototxt = './caffenet.prototxt'
caffemodel = './caffenet.caffemodel'
TEST_ITER=300

In [2]:
def test_accuracy(net, test_iter):
    loss = np.zeros(1)
    top1 = np.zeros(1)

    test_loss = 0
    top1_accuracy = 0

    for test_it in range(test_iter):
        net.forward()
        test_loss += net.blobs['loss'].data
        top1_accuracy += net.blobs['accuracy'].data  

    loss[0] = test_loss / test_iter 
    top1[0] = top1_accuracy / test_iter

    print 'Loss: ', loss[0], ', Accuracy Top1:', top1[0]
    log.writelines('Loss: '+ str(loss[0]) + ', Accuracy Top1:' + str(top1[0]) + '\n')
    return loss[0], top1[0]

def prune_fm(prototxt, caffemodel, layer, iteration):#剪枝哪一层，这一层有几个fm，每次简直测试跑多少iteration
    net = caffe.Net(prototxt, caffemodel, caffe.TEST)

    zeros = np.zeros(net.params[layer][0].data[0].shape)#用于剪枝
    loss = np.zeros(net.blobs[layer].channels)
    top1 = np.zeros(net.blobs[layer].channels)
    
    for i in range(net.blobs[layer].channels):
        net.copy_from(caffemodel)
        net.params[layer][0].data[i] = zeros
        print 'pruning', layer, 'feature map',i, '...'
        log.writelines('pruning ' + layer + ' feature map ' + str(i) + '...\n')
        loss[i],top1[i] = test_accuracy(net, iteration)
    return loss,top1

def test_pruned_fm(prototxt, caffemodel, layer, iteration, prune_rate):
    
    net = caffe.Net(prototxt, caffemodel, caffe.TEST)

    zeros=np.zeros(net.params[layer][0].data[0].shape)
    loss = np.zeros(net.blobs[layer].channels)
    top1 = np.zeros(net.blobs[layer].channels)
    
    prune_cnt = int(prune_rate*net.blobs[layer].channels)
    
    [loss,top1]=prune_fm(prototxt, caffemodel, layer, iteration)
    
    print 'cut unimportant features...'
    log.writelines('cut unimportant features... \n')
    #重要程度越低越靠前
    loss_index = np.argsort(loss)
    top1_index = np.argsort(-top1)
    
    #根据top1做剪枝
    print 'prune basing on top1...'
    log.writelines('prune basing on top1... \n')
    print 'fm to be pruned: ', top1_index[:prune_cnt]
    log.writelines('fm to be pruned: '+ str(top1_index[:prune_cnt])+'\n')           
    net = caffe.Net(prototxt, caffemodel, caffe.TEST)
    for i in range(prune_cnt):
        net.params[layer][0].data[top1_index[i]]=zeros
    test_accuracy(net, TEST_ITER)
    
    #根据loss做剪枝
    print 'prune basing on loss...'
    log.writelines('prune basing on loss... \n')
    print 'fm to be pruned:', loss_index[:prune_cnt]
    log.writelines('fm to be pruned: '+ str(loss_index[:prune_cnt])+'\n')       
    net = caffe.Net(prototxt, caffemodel, caffe.TEST)
    for i in range(prune_cnt):
        net.params[layer][0].data[loss_index[i]]=zeros
    test_accuracy(net, TEST_ITER)
    
    print 'cut important features...'
    log.writelines('prune basing on top1... \n')
    #重要程度越低越靠前
    loss_index = np.argsort(-loss)
    top1_index = np.argsort(top1)
    
    #根据top1做剪枝
    print 'prune basing on top1...'
    log.writelines('prune basing on top1... \n')
    print 'fm to be pruned:', top1_index[:prune_cnt]
    log.writelines('fm to be pruned: '+ str(top1_index[:prune_cnt])+'\n') 
    net = caffe.Net(prototxt, caffemodel, caffe.TEST)
    for i in range(prune_cnt):
        net.params[layer][0].data[top1_index[i]]=zeros
    test_accuracy(net, TEST_ITER)
    
    #根据loss做剪枝
    print 'prune basing on loss...'
    log.writelines('prune basing on top1... \n')
    print 'fm to be pruned:', loss_index[:prune_cnt]
    log.writelines('fm to be pruned: '+ str(loss_index[:prune_cnt])+'\n') 
    net = caffe.Net(prototxt, caffemodel, caffe.TEST)
    for i in range(prune_cnt):
        net.params[layer][0].data[loss_index[i]]=zeros
    test_accuracy(net, TEST_ITER)

In [3]:
log=open("CaffeNet.log",'a+') 
caffenet = caffe.Net(prototxt, caffemodel, caffe.TEST)
test_accuracy(caffenet, 300)
log.close()

Loss:  1.8837960358460744 , Accuracy Top1: 0.5640000030398369


In [4]:
log=open("CaffeNet.log",'a+') 
test_pruned_fm(prototxt, caffemodel, 'conv1', 100, 0.125)
log.close()

pruning conv1 feature map 0 ...


KeyboardInterrupt: 

In [13]:
log=open("CaffeNet.log",'a+') 
test_pruned_fm(prototxt, caffemodel, 'conv2', 100, 0.125)
test_pruned_fm(prototxt, caffemodel, 'conv3', 100, 0.125)
log.close()

pruning conv2 feature map 0 ...
Loss:  1.9349475610256195 , Accuracy Top1: 0.5592000022530556
pruning conv2 feature map 1 ...
Loss:  1.8544706678390503 , Accuracy Top1: 0.5644000002741814
pruning conv2 feature map 2 ...
Loss:  1.8897296953201295 , Accuracy Top1: 0.5608000013232232
pruning conv2 feature map 3 ...
Loss:  1.8295962357521056 , Accuracy Top1: 0.5680000022053718
pruning conv2 feature map 4 ...
Loss:  1.8436152231693268 , Accuracy Top1: 0.5732000023126602
pruning conv2 feature map 5 ...
Loss:  1.9035700857639313 , Accuracy Top1: 0.5608000028133392
pruning conv2 feature map 6 ...
Loss:  1.885556122660637 , Accuracy Top1: 0.5608000013232232
pruning conv2 feature map 7 ...
Loss:  1.8517199897766112 , Accuracy Top1: 0.5696000006794929
pruning conv2 feature map 8 ...
Loss:  1.9225409173965453 , Accuracy Top1: 0.5596000012755394
pruning conv2 feature map 9 ...
Loss:  1.8462227916717528 , Accuracy Top1: 0.5744000032544136
pruning conv2 feature map 10 ...
Loss:  1.9399869513511658 , 

Loss:  1.843254210948944 , Accuracy Top1: 0.5715999993681907
pruning conv2 feature map 88 ...
Loss:  1.917495777606964 , Accuracy Top1: 0.5596000018715859
pruning conv2 feature map 89 ...
Loss:  1.9177360391616822 , Accuracy Top1: 0.5632000029087066
pruning conv2 feature map 90 ...
Loss:  1.9521052396297456 , Accuracy Top1: 0.5526000002026558
pruning conv2 feature map 91 ...
Loss:  1.853893619775772 , Accuracy Top1: 0.5637999996542931
pruning conv2 feature map 92 ...
Loss:  1.8895765674114227 , Accuracy Top1: 0.5642000025510788
pruning conv2 feature map 93 ...
Loss:  1.8275659251213074 , Accuracy Top1: 0.5668000033497811
pruning conv2 feature map 94 ...
Loss:  1.8430526649951935 , Accuracy Top1: 0.5720000010728836
pruning conv2 feature map 95 ...
Loss:  1.9092296957969666 , Accuracy Top1: 0.5576000007987022
pruning conv2 feature map 96 ...
Loss:  1.8826387059688567 , Accuracy Top1: 0.5642000034451484
pruning conv2 feature map 97 ...
Loss:  1.8471499168872834 , Accuracy Top1: 0.56999999

Loss:  1.9963187162081402 , Accuracy Top1: 0.5382000009218851
cut important features...
prune basing on top1...
fm to be pruned: [172  55  90  75 175 200 170 100 108 190 215 240  10  78  38  95 120  65
  18 178 135  70  85 155  40 218 235 130 205 146 115 228]
Loss:  2.485195633570353 , Accuracy Top1: 0.4484666649500529
prune basing on loss...
fm to be pruned: [172  90 200 100  70 120 170 130 190  10  50  30 160  40 230 240   0 150
 110 140  60 210  80 108  20 220 250 180  78  38 138  58]
Loss:  2.3364774147669474 , Accuracy Top1: 0.47593333234389623
pruning conv3 feature map 0 ...
Loss:  1.9351314830780029 , Accuracy Top1: 0.5608000022172928
pruning conv3 feature map 1 ...
Loss:  1.8424649155139923 , Accuracy Top1: 0.566000000834465
pruning conv3 feature map 2 ...
Loss:  1.878607940673828 , Accuracy Top1: 0.5650000047683715
pruning conv3 feature map 3 ...
Loss:  1.8323431253433227 , Accuracy Top1: 0.5678000035881996
pruning conv3 feature map 4 ...
Loss:  1.8415843677520751 , Accuracy T

Loss:  1.8413304579257965 , Accuracy Top1: 0.5694000026583672
pruning conv3 feature map 82 ...
Loss:  1.8797086119651794 , Accuracy Top1: 0.5626000034809112
pruning conv3 feature map 83 ...
Loss:  1.8295767986774445 , Accuracy Top1: 0.5694000017642975
pruning conv3 feature map 84 ...
Loss:  1.843256242275238 , Accuracy Top1: 0.5684000006318093
pruning conv3 feature map 85 ...
Loss:  1.9049852228164672 , Accuracy Top1: 0.5590000006556511
pruning conv3 feature map 86 ...
Loss:  1.8884762805700301 , Accuracy Top1: 0.5618000003695488
pruning conv3 feature map 87 ...
Loss:  1.8427656996250152 , Accuracy Top1: 0.5738000014424324
pruning conv3 feature map 88 ...
Loss:  1.91860560297966 , Accuracy Top1: 0.5602000021934509
pruning conv3 feature map 89 ...
Loss:  1.8403459513187408 , Accuracy Top1: 0.5754000049829483
pruning conv3 feature map 90 ...
Loss:  1.932526569366455 , Accuracy Top1: 0.5598000022768974
pruning conv3 feature map 91 ...
Loss:  1.8392635953426362 , Accuracy Top1: 0.568200002

Loss:  1.8452696359157563 , Accuracy Top1: 0.5714000007510185
pruning conv3 feature map 168 ...
Loss:  1.924102486371994 , Accuracy Top1: 0.5580000022053718
pruning conv3 feature map 169 ...
Loss:  1.836080870628357 , Accuracy Top1: 0.5770000046491623
pruning conv3 feature map 170 ...
Loss:  1.9302814960479737 , Accuracy Top1: 0.5626000022888183
pruning conv3 feature map 171 ...
Loss:  1.8443210899829865 , Accuracy Top1: 0.5646000030636787
pruning conv3 feature map 172 ...
Loss:  1.8863586723804473 , Accuracy Top1: 0.5630000042915344
pruning conv3 feature map 173 ...
Loss:  1.8272446477413178 , Accuracy Top1: 0.5670000028610229
pruning conv3 feature map 174 ...
Loss:  1.8417399990558625 , Accuracy Top1: 0.5702000039815903
pruning conv3 feature map 175 ...
Loss:  1.9028474855422974 , Accuracy Top1: 0.557800001502037
pruning conv3 feature map 176 ...
Loss:  1.8895224714279175 , Accuracy Top1: 0.563000003695488
pruning conv3 feature map 177 ...
Loss:  1.844414484500885 , Accuracy Top1: 0.

Loss:  1.8294648325443268 , Accuracy Top1: 0.5698000049591064
pruning conv3 feature map 254 ...
Loss:  1.8444325530529022 , Accuracy Top1: 0.5706000021100044
pruning conv3 feature map 255 ...
Loss:  1.903820391893387 , Accuracy Top1: 0.5580000001192092
pruning conv3 feature map 256 ...
Loss:  1.8847763401269912 , Accuracy Top1: 0.5644000035524368
pruning conv3 feature map 257 ...
Loss:  1.845713472366333 , Accuracy Top1: 0.571800001859665
pruning conv3 feature map 258 ...
Loss:  1.9163112843036652 , Accuracy Top1: 0.5600000011920929
pruning conv3 feature map 259 ...
Loss:  1.8384151363372803 , Accuracy Top1: 0.5774000024795533
pruning conv3 feature map 260 ...
Loss:  1.9486111605167389 , Accuracy Top1: 0.5578000012040139
pruning conv3 feature map 261 ...
Loss:  1.842940957546234 , Accuracy Top1: 0.565
pruning conv3 feature map 262 ...
Loss:  1.882145255804062 , Accuracy Top1: 0.5650000047683715
pruning conv3 feature map 263 ...
Loss:  1.8240539669990539 , Accuracy Top1: 0.5700000041723

Loss:  1.840319538116455 , Accuracy Top1: 0.5728000032901764
pruning conv3 feature map 340 ...
Loss:  1.9333558559417725 , Accuracy Top1: 0.5618000009655952
pruning conv3 feature map 341 ...
Loss:  1.847884750366211 , Accuracy Top1: 0.563000001013279
pruning conv3 feature map 342 ...
Loss:  1.8867303466796874 , Accuracy Top1: 0.5620000034570694
pruning conv3 feature map 343 ...
Loss:  1.8277394711971282 , Accuracy Top1: 0.5682000046968461
pruning conv3 feature map 344 ...
Loss:  1.8463661861419678 , Accuracy Top1: 0.5688000029325485
pruning conv3 feature map 345 ...
Loss:  1.9085325813293457 , Accuracy Top1: 0.5610000032186508
pruning conv3 feature map 346 ...
Loss:  1.8954374241828917 , Accuracy Top1: 0.5630000022053718
pruning conv3 feature map 347 ...
Loss:  1.857124069929123 , Accuracy Top1: 0.5674000015854835
pruning conv3 feature map 348 ...
Loss:  1.919447170495987 , Accuracy Top1: 0.560600001513958
pruning conv3 feature map 349 ...
Loss:  1.8438840556144713 , Accuracy Top1: 0.5

In [3]:
log=open("CaffeNet.log",'a+') 
test_pruned_fm(prototxt, caffemodel, 'conv4', 200, 0.125)
test_pruned_fm(prototxt, caffemodel, 'conv5', 200, 0.125)
log.close()

pruning conv4 feature map 0 ...
Loss:  1.8895195400714875 , Accuracy Top1: 0.5629000018537045
pruning conv4 feature map 1 ...
Loss:  1.8537633603811263 , Accuracy Top1: 0.5663000032305717
pruning conv4 feature map 2 ...
Loss:  1.880088809132576 , Accuracy Top1: 0.5630000025033951
pruning conv4 feature map 3 ...
Loss:  1.8588337215781212 , Accuracy Top1: 0.5672000008821487
pruning conv4 feature map 4 ...
Loss:  1.8768933588266372 , Accuracy Top1: 0.5665000015497208
pruning conv4 feature map 5 ...
Loss:  1.8878141558170318 , Accuracy Top1: 0.5630000033974647
pruning conv4 feature map 6 ...
Loss:  1.8556404292583466 , Accuracy Top1: 0.5655000028014183
pruning conv4 feature map 7 ...
Loss:  1.8735169851779938 , Accuracy Top1: 0.5633000007271767
pruning conv4 feature map 8 ...
Loss:  1.8614615294337273 , Accuracy Top1: 0.5672000031173229
pruning conv4 feature map 9 ...
Loss:  1.877451998591423 , Accuracy Top1: 0.5680000038444996
pruning conv4 feature map 10 ...
Loss:  1.8857287412881851 , A

Loss:  1.87095858335495 , Accuracy Top1: 0.5652000026404858
pruning conv4 feature map 88 ...
Loss:  1.8637675887346268 , Accuracy Top1: 0.5680000014603138
pruning conv4 feature map 89 ...
Loss:  1.878441948890686 , Accuracy Top1: 0.5678000026941299
pruning conv4 feature map 90 ...
Loss:  1.8879033863544463 , Accuracy Top1: 0.5643000008165836
pruning conv4 feature map 91 ...
Loss:  1.8588331973552703 , Accuracy Top1: 0.5654000030457973
pruning conv4 feature map 92 ...
Loss:  1.8732640641927718 , Accuracy Top1: 0.5649000020325183
pruning conv4 feature map 93 ...
Loss:  1.8627862811088562 , Accuracy Top1: 0.5682000023126602
pruning conv4 feature map 94 ...
Loss:  1.8781116729974747 , Accuracy Top1: 0.5666000029444694
pruning conv4 feature map 95 ...
Loss:  1.8863193678855896 , Accuracy Top1: 0.5622000034153462
pruning conv4 feature map 96 ...
Loss:  1.8536339056491853 , Accuracy Top1: 0.5652000033855438
pruning conv4 feature map 97 ...
Loss:  1.8725085496902465 , Accuracy Top1: 0.56460000

Loss:  1.8636736792325974 , Accuracy Top1: 0.5690000024437905
pruning conv4 feature map 174 ...
Loss:  1.8787827050685884 , Accuracy Top1: 0.5687000021338463
pruning conv4 feature map 175 ...
Loss:  1.8868989259004594 , Accuracy Top1: 0.5630000041425228
pruning conv4 feature map 176 ...
Loss:  1.8542088103294372 , Accuracy Top1: 0.5672000041604042
pruning conv4 feature map 177 ...
Loss:  1.8739215195178986 , Accuracy Top1: 0.5651000013947487
pruning conv4 feature map 178 ...
Loss:  1.862810958325863 , Accuracy Top1: 0.5690000033378602
pruning conv4 feature map 179 ...
Loss:  1.875770633816719 , Accuracy Top1: 0.5682000035047531
pruning conv4 feature map 180 ...
Loss:  1.8872987776994705 , Accuracy Top1: 0.5638000029325485
pruning conv4 feature map 181 ...
Loss:  1.8553220564126969 , Accuracy Top1: 0.5654000033438206
pruning conv4 feature map 182 ...
Loss:  1.8739961373806 , Accuracy Top1: 0.565200001746416
pruning conv4 feature map 183 ...
Loss:  1.860069741308689 , Accuracy Top1: 0.56

Loss:  1.8841453313827514 , Accuracy Top1: 0.5648000015318394
pruning conv4 feature map 260 ...
Loss:  1.891216636300087 , Accuracy Top1: 0.5602000008523464
pruning conv4 feature map 261 ...
Loss:  1.8536710059642791 , Accuracy Top1: 0.5658000023663043
pruning conv4 feature map 262 ...
Loss:  1.8810751402378083 , Accuracy Top1: 0.5626000027358532
pruning conv4 feature map 263 ...
Loss:  1.8625109708309173 , Accuracy Top1: 0.5683000008761883
pruning conv4 feature map 264 ...
Loss:  1.8799924814701081 , Accuracy Top1: 0.5677000035345554
pruning conv4 feature map 265 ...
Loss:  1.8931070560216903 , Accuracy Top1: 0.5627000015974045
pruning conv4 feature map 266 ...
Loss:  1.8536109471321105 , Accuracy Top1: 0.5661000041663646
pruning conv4 feature map 267 ...
Loss:  1.8776401925086974 , Accuracy Top1: 0.565700002014637
pruning conv4 feature map 268 ...
Loss:  1.8693327021598816 , Accuracy Top1: 0.5651000007987023
pruning conv4 feature map 269 ...
Loss:  1.8841409957408906 , Accuracy Top1:

pruning conv4 feature map 345 ...
Loss:  1.891252567768097 , Accuracy Top1: 0.5638000011444092
pruning conv4 feature map 346 ...
Loss:  1.8506942349672317 , Accuracy Top1: 0.5665000030398368
pruning conv4 feature map 347 ...
Loss:  1.8773971199989319 , Accuracy Top1: 0.5627000026404858
pruning conv4 feature map 348 ...
Loss:  1.8624850857257842 , Accuracy Top1: 0.5674000023305417
pruning conv4 feature map 349 ...
Loss:  1.8797226220369339 , Accuracy Top1: 0.5675000037252903
pruning conv4 feature map 350 ...
Loss:  1.8895460653305054 , Accuracy Top1: 0.5636000034213066
pruning conv4 feature map 351 ...
Loss:  1.8544164484739303 , Accuracy Top1: 0.5662000031769275
pruning conv4 feature map 352 ...
Loss:  1.8745028728246689 , Accuracy Top1: 0.5640000027418136
pruning conv4 feature map 353 ...
Loss:  1.865689074099064 , Accuracy Top1: 0.5671000029146671
pruning conv4 feature map 354 ...
Loss:  1.8817610573768615 , Accuracy Top1: 0.5671000027656555
pruning conv4 feature map 355 ...
Loss:  1

Loss:  1.8770367777347565 , Accuracy Top1: 0.5676000024378299
pruning conv5 feature map 35 ...
Loss:  1.8887858873605727 , Accuracy Top1: 0.5639000025391578
pruning conv5 feature map 36 ...
Loss:  1.8588670778274536 , Accuracy Top1: 0.5655000047385692
pruning conv5 feature map 37 ...
Loss:  1.8676753968000412 , Accuracy Top1: 0.5645000030100346
pruning conv5 feature map 38 ...
Loss:  1.864836829006672 , Accuracy Top1: 0.5667000019550323
pruning conv5 feature map 39 ...
Loss:  1.879046271443367 , Accuracy Top1: 0.5674000027775764
pruning conv5 feature map 40 ...
Loss:  1.887639342546463 , Accuracy Top1: 0.5629000037908554
pruning conv5 feature map 41 ...
Loss:  1.8537993294000625 , Accuracy Top1: 0.566200003027916
pruning conv5 feature map 42 ...
Loss:  1.8739105862379075 , Accuracy Top1: 0.5658000013232232
pruning conv5 feature map 43 ...
Loss:  1.863361845612526 , Accuracy Top1: 0.5671000027656555
pruning conv5 feature map 44 ...
Loss:  1.8817288285493852 , Accuracy Top1: 0.5673000021

pruning conv5 feature map 121 ...
Loss:  1.8567697709798814 , Accuracy Top1: 0.5633000038564205
pruning conv5 feature map 122 ...
Loss:  1.8731736993789674 , Accuracy Top1: 0.5644000011682511
pruning conv5 feature map 123 ...
Loss:  1.8606639558076858 , Accuracy Top1: 0.5677000017464161
pruning conv5 feature map 124 ...
Loss:  1.8787095957994462 , Accuracy Top1: 0.56690000385046
pruning conv5 feature map 125 ...
Loss:  1.8882830530405044 , Accuracy Top1: 0.563400002270937
pruning conv5 feature map 126 ...
Loss:  1.8590756946802138 , Accuracy Top1: 0.5663000024855137
pruning conv5 feature map 127 ...
Loss:  1.873961837887764 , Accuracy Top1: 0.56430000141263
pruning conv5 feature map 128 ...
Loss:  1.8722085666656494 , Accuracy Top1: 0.5650000008940697
pruning conv5 feature map 129 ...
Loss:  1.890990853905678 , Accuracy Top1: 0.56490000218153
pruning conv5 feature map 130 ...
Loss:  1.8920913779735564 , Accuracy Top1: 0.5614000007510185
pruning conv5 feature map 131 ...
Loss:  1.862642

Loss:  1.8778177988529205 , Accuracy Top1: 0.5635000020265579
pruning conv5 feature map 208 ...
Loss:  1.8647955191135406 , Accuracy Top1: 0.5674000039696694
pruning conv5 feature map 209 ...
Loss:  1.8824486279487609 , Accuracy Top1: 0.5675000020861626
pruning conv5 feature map 210 ...
Loss:  1.8877850699424743 , Accuracy Top1: 0.5634000018239022
pruning conv5 feature map 211 ...
Loss:  1.8629774576425553 , Accuracy Top1: 0.5653000038862228
pruning conv5 feature map 212 ...
Loss:  1.8783454066514969 , Accuracy Top1: 0.5646000017225742
pruning conv5 feature map 213 ...
Loss:  1.8695346441864968 , Accuracy Top1: 0.5682000017166138
pruning conv5 feature map 214 ...
Loss:  1.8791204237937926 , Accuracy Top1: 0.5674000023305417
pruning conv5 feature map 215 ...
Loss:  1.8904357063770294 , Accuracy Top1: 0.5636000022292137
pruning conv5 feature map 216 ...
Loss:  1.85886634349823 , Accuracy Top1: 0.5645000027120113
pruning conv5 feature map 217 ...
Loss:  1.8761569511890412 , Accuracy Top1:

In [4]:
log=open("CaffeNet.log",'a+') 
test_pruned_fm(prototxt, caffemodel, 'conv6', 200, 0.125)
test_pruned_fm(prototxt, caffemodel, 'conv7', 200, 0.125)
test_pruned_fm(prototxt, caffemodel, 'conv8', 200, 0.125)
log.close()

KeyError: 'conv6'